In [ ]:
# Code to Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Code to Install gspread
!pip install --upgrade gspread

!pip install ultralytics

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/StrongSORT-YOLO/yolov5


/content/drive/MyDrive/StrongSORT-YOLO/yolov5


In [ ]:
cd yolov5


/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5


In [ ]:
!pip install -r requirements.txt  # install

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Authenticate and authorize
creds, _ = default()

gc = gspread.authorize(creds)

# Open the Google Sheet by URL and select the worksheet
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UJ6in71M5X_oDXmKOL2tYhlW21vWqPjBZYAISXTuDHI/edit#gid=1725335115').worksheet('LIVE SUBMISSIONS')

# Get all values from the worksheet
values = worksheet.get_all_values()

# Initialize a flag to check if we found a matching row
found_matching_row = False

# Loop through the rows and check the "AA" column
for index, row in enumerate(values[1:], start=2):  # Start from the second row (index 1)
    aa_value = row[26]  # Assuming "AA" column is the 27th column (0-based index)

    if not aa_value:  # If "AA" column is empty
        email_id = row[3]  # Get email id from column D (0-based index)
        url_link = row[11]  # Get URL link from column L (0-based index)

        # Print the information for the current row
        print(f"Row Index: {index}, Email ID: {email_id}, URL Link: {url_link}")

        # Set the flag to True to indicate that we found a matching row
        found_matching_row = True

        # Break out of the loop after processing the first matching row
        break

# Check if no matching rows were found
if not found_matching_row:
    print("No rows with empty 'AA' column found.")


No rows with empty 'AA' column found.


In [ ]:
import os
import subprocess
import requests

# Use the specific YOLO directory path on your Google Drive
yolo_directory = '/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/'

# Create a directory to store downloaded videos
download_directory = os.path.join(yolo_directory, 'downloaded_videos')
os.makedirs(download_directory, exist_ok=True)

try:
    # Print the fetched URL
    print(f"Processing video from URL: {url_link}")

    # Extract the video filename from the URL (assuming the URL ends with .mp4)
    video_filename = os.path.basename(url_link)

    # Define the full path to save the downloaded video
    video_filepath = os.path.join(download_directory, video_filename)

    # Download the video from the URL
    response = requests.get(url_link)
    with open(video_filepath, 'wb') as video_file:
        video_file.write(response.content)

    # Define the output directory for YOLO results
    output_directory = os.path.join(yolo_directory, 'yolo_result')
    os.makedirs(output_directory, exist_ok=True)

    # Print video file path
    print(f"Video file path is: {video_filepath}")

    # Define the YOLO command
    yolo_command = f'python {os.path.join(yolo_directory, "detect.py")} --source "{video_filepath}" --weights "{os.path.join(yolo_directory, "best.pt")}" --project "{output_directory}"'

    print(f"YOLO command is: {yolo_command}")

    # Execute the YOLO command using subprocess
    subprocess.run(yolo_command, shell=True)

except Exception as e:
    print(f"Error processing {last_video_url}: {str(e)}")


Processing video from URL: https://s3.amazonaws.com/pf-user-files-01/u-54188/uploads/2023-09-14/cf34pru/00e35688-8355-459d-9e81-8935f15058f3.mp4
Video file path is: /content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/downloaded_videos/00e35688-8355-459d-9e81-8935f15058f3.mp4
YOLO command is: python /content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/detect.py --source "/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/downloaded_videos/00e35688-8355-459d-9e81-8935f15058f3.mp4" --weights "/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/best.pt" --project "/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/yolo_result"


In [ ]:
import os
import glob

# Define the directory path
directory_path = '/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/yolo_result/exp'

# Get a list of all files in the directory
files = glob.glob(os.path.join(directory_path, '*'))

# Sort the files by modification time (most recent first)
files.sort(key=os.path.getmtime, reverse=True)

# Check if there are any files in the directory
if files:
    # Get the most recently added file
    most_recent_file = files[0]

    # Extract the file name from the path
    file_name = os.path.basename(most_recent_file)

    # Print the most recently added file name
    print(f"Most recently added file: {file_name}")
else:
    print("The directory is empty.")


Most recently added file: 00e35688-8355-459d-9e81-8935f15058f3.mp4


In [ ]:
import os
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime

# Set the path to your JSON credentials file
credentials_file = '/content/drive/MyDrive/StrongSORT-YOLO/yolov5/yolov5/elated-coral-398820-92ee6ef533d7.json'

# Initialize the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_file, scopes=['https://www.googleapis.com/auth/drive.readonly']
)

# Build the Drive API client
drive_service = build('drive', 'v3', credentials=credentials)

folder_id = '1-lLXvWu-_JNMeolT2qSw2_b7axcAJ7Zn'

# List all the files in the folder and sort them by modified time
results = drive_service.files().list(
    q=f"'{folder_id}' in parents",
    fields='files(id, name, modifiedTime)',
    orderBy='modifiedTime desc',
).execute()

# Get the first (most recent) file from the list
if 'files' in results:
    most_recent_file = results['files'][0]

    # Get the file's metadata
    file_id = most_recent_file['id']
    file_metadata = drive_service.files().get(fileId=file_id, fields='webViewLink').execute()

    # Extract the sharable link from the metadata
    sharable_link = file_metadata.get('webViewLink')

    print("Most Recent File Sharable Link:", sharable_link)
else:
    print("No files found in the folder.")


Most Recent File Sharable Link: https://drive.google.com/file/d/1-7H_ZoV3Snki2lCaJMAmL2D8XCK8WUbQ/view?usp=drivesdk


In [ ]:
import smtplib

sender_email = 'studentsheart.es@gmail.com'
receiver_email = email_id
sharable_link = sharable_link

# Format the email message with the sharable link
message = f"""\
Subject: Email Test from SohailAIExpert

New entry and the processed Video file can be accessed at this Link:
{sharable_link}
"""

# Use the App Password you generated for your Gmail account
app_password = 'mllt skxs rflb uueg'  # Replace with your App Password

try:
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, receiver_email, message)
    print(f"Successfully sent email to : {email_id}")
except smtplib.SMTPException as e:
    print(f"Error: unable to send email\n{e}")


Successfully sent email to : isaac.cohen1@gmail.com


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Authenticate and authorize
creds, _ = default()

# Authorize the Google Sheets API
gc = gspread.authorize(creds)

# Open the sheet and select the tab
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1UJ6in71M5X_oDXmKOL2tYhlW21vWqPjBZYAISXTuDHI/edit#gid=1725335115'
worksheet_name = 'LIVE SUBMISSIONS'
worksheet = gc.open_by_url(spreadsheet_url).worksheet(worksheet_name)

# Define the row number and new value
row_number = index
new_value = sharable_link

# Get the last column (AA) letter
last_column = 'AA'

# Define the cell to update in AA column
cell_to_update = f'{last_column}{row_number}'

# Update the specific cell in AA column with the new value
worksheet.update(cell_to_update, new_value)

print(f"Updated cell {cell_to_update} with '{new_value}'.")


Updated cell AA16 with 'https://drive.google.com/file/d/1-7H_ZoV3Snki2lCaJMAmL2D8XCK8WUbQ/view?usp=drivesdk'.
